# Import modules, libraries, and packages

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Get the current working directory of the notebook
CURRENT_DIR = os.getcwd()
# Add the parent directory of the notebook to the Python path
ROOT_DIR = os.path.abspath(os.path.join(CURRENT_DIR, '..', '..', '..'))
sys.path.append(ROOT_DIR)

import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import geopandas as gpd
from fiona.crs import from_epsg

from utils.utils import load_util_data, get_unusable_basins

# Parameters and constants

In [2]:
data_dir, data_gen = load_util_data(CURRENT_DIR)

In [3]:
# FILES_DIR = os.path.join(root_dir, 'gladwell/hydrology/SUMMA/summa-ml-models')

DATASET = 'CAMELS_spat_NH'

COUNTRY = 'CAN'  # 'USA' or 'CAN'

INPUT_FILES_DIR = os.path.join(data_dir['summa_ml_models_dir'], DATASET)
# ATTR_FILES_DIR = os.path.join(FILES_DIR, DATASET.upper(), 'camels_attributes_v2.0')
BASIN_SET_FILES_DIR = os.path.join(INPUT_FILES_DIR, 'merged_lumped_shapes')

# FORCINGS = ['daymet', 'nldas', 'maurer']
# FORCINGS_dict = {
#     'daymet': 'cida',
#     'nldas': 'nldas',
#     'maurer': 'maurer',
# }

# HRU_IDS = [
#     'CAN_01AD002',
#     'CAN_01AL003',
#     'CAN_01AF007',
#     'CAN_01AJ003',
#     'CAN_01AJ004',
#     'CAN_01AJ010',
#     'CAN_01AK001',
#     'CAN_01AK006',
#     'CAN_01AK007',
#     'CAN_01AL002'
# ]

## Load unusable basins

In [4]:
unusuable_basins = get_unusable_basins(INPUT_FILES_DIR, data_gen['camels_spat_unusable'])

## Loading basin IDs

In [5]:
##Read the basin shapefile
gdf_basins = gpd.read_file(os.path.join(BASIN_SET_FILES_DIR, 'merged_lumped_outlines.shp'))
# Subset the basins to only include the ones that are usable
gdf_basins = gdf_basins[~gdf_basins['Station_id'].isin(unusuable_basins)]

# # merged_lumped_outlines
# camels_new_delineation = gpd.read_file(os.path.join(BASIN_SET_FILES_DIR, 'camels_new_delineation.shp'))

gdf_borders = gpd.read_file(os.path.join(INPUT_FILES_DIR, 'NA_PoliticalDivision/boundaries_p_2021_v3.shp'))
states = gpd.read_file(os.path.join(BASIN_SET_FILES_DIR, 'USA_Canada_ShapefileMerge.shp'))

In [6]:
gdf_basins

,Country,Station_id,Station_na,Delineatio,Delineat_1,geometry
1,CAN,01AD003,ST. FRANCIS RIVER AT OUTLET OF GLASIER LAKE,high,None,"MULTIPOLYGON (((-69.04125 47.21125, -69.04208 ..."
3,CAN,01AF007,GRANDE RIVIERE AT VIOLETTE BRIDGE,high,None,"POLYGON ((-67.67625 47.45375, -67.67542 47.453..."
4,CAN,01AF009,IROQUOIS RIVER AT MOULIN MORNEAULT,high,None,"POLYGON ((-68.38875 47.68708, -68.38375 47.687..."
5,CAN,01AJ003,MEDUXNEKEAG RIVER NEAR BELLEVILLE,high,None,"MULTIPOLYGON (((-67.97292 46.45792, -67.97375 ..."
6,CAN,01AJ004,BIG PRESQUE ISLE STREAM AT TRACEY MILLS,high,None,"POLYGON ((-67.91625 46.72208, -67.91458 46.722..."
...,...,...,...,...,...,...
1693,USA,14309500,"WEST FORK COW CREEK NEAR GLENDALE, OR",high,None,"POLYGON ((-123.71042 42.90458, -123.70958 42.9..."
1694,USA,14316700,"STEAMBOAT CREEK NEAR GLIDE, OR",high,None,"POLYGON ((-122.60542 43.60042, -122.60375 43.6..."
1695,USA,14325000,"SOUTH FORK COQUILLE RIVER AT POWERS, OR",high,None,"POLYGON ((-124.02792 42.90625, -124.02708 42.9..."
1696,USA,14362250,"STAR GULCH NEAR RUCH, OR.",high,None,"POLYGON ((-123.15292 42.19625, -123.15208 42.1..."


In [7]:
# Set the CRS for gdf_basins
gdf_basins.crs = from_epsg(4326)

# Set a common CRS for both GeoDataFrames
common_crs = 'ESRI:102008'
gdf_basins = gdf_basins.to_crs(common_crs)

gdf_borders = gdf_borders.to_crs(common_crs)
gdf_borders = gdf_borders.set_crs(common_crs, allow_override=True)

In [8]:
gdf_borders = gdf_borders.to_crs(common_crs)
gdf_borders = gdf_borders.set_crs(common_crs, allow_override=True)

# Catchments plots

In [14]:
# Map color for country (CAN, USA)
cmap = {'CAN': 'r', 'USA': 'b'}

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the reprojected gdf_borders
# gdf_borders.boundary.plot(ax=ax, color=None, edgecolor='k', linewidth=0.1)
gdf_borders.plot(ax=ax, facecolor='0.2',edgecolor='0.7', linewidth=0.1)

# # Plot the merged_lumped_outlines
# gdf_basins.plot(ax=ax, color='lightblue', edgecolor='gray', linewidth=0.1)
# Plot the basins with different colors for each country
for country, color in cmap.items():
    gdf_basins[gdf_basins['Country'] == country].plot(ax=ax, color=color, edgecolor='k', linewidth=0.1)

# Chart junk
ax.set_xlim([-2.5*1e6, 3.1*1e6])
ax.set_ylim([-1.6*1e6, 3.7*1e6])

# ax.set_title("CAMELS_spat catchments")
# ax.set_xlabel('Longitude [degrees East]')
# ax.set_ylabel('Latitude [degrees North]')

# Remove x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])

# Show the plot
plt.show()